In [33]:
import requests
import json
from pprint import pprint


In [34]:
BASE_URL = "http://localhost:8000"

def call_api(method, endpoint, **kwargs):
    url = f"{BASE_URL}{endpoint}"
    try:
        resp = requests.request(
            method,
            url,
            timeout=5,   # prevents hanging
            **kwargs
        )
        pprint(resp.json())
        return resp
    except requests.exceptions.ConnectionError:
        print(" FastAPI service is NOT running")
        print(" Start it using:")
        print("   uvicorn app.main:app --reload")
    except requests.exceptions.Timeout:
        print("Request timed out")
    except Exception as e:
        print("Error:", e)
        return None


Base Configuration

In [35]:
BASE_URL = "http://localhost:8000"

def pretty(response):
    try:
        pprint(response.json())
    except:
        print(response.text)


SYSTEM HEALTH TESTS

In [5]:
resp = requests.get(f"{BASE_URL}/api/health")
pretty(resp)


{'compliance_enabled': True,
 'status': 'healthy',
 'timestamp': '2025-12-17T08:47:05.850369',
 'version': '1.0.0'}


Compliance Status (Initial)

In [7]:
resp = requests.get(f"{BASE_URL}/api/compliance/status")
pretty(resp)


{'active_agents': ['AccessControlAgent',
                   'PrivacyAgent',
                   'OutputGuardAgent',
                   'AuditAgent'],
 'enabled': True,
 'enforced_regulations': ['GDPR', 'HIPAA', 'PCI-DSS', 'AI Act'],
 'updated_at': '2025-12-17T07:31:06.219194'}


NON-COMPLIANT MODE (BYPASS)

In [79]:
payload = {
    "input_text": "Patient with Chakravarthy Nara has chest pain and dizziness",
    "request_type": "patient_triage",
    "user_role": "clinician"
}

resp = requests.post(f"{BASE_URL}/api/process", json=payload)
pretty(resp)


{'access_granted': True,
 'audit_id': 'AUDIT-20251217-201234',
 'audit_log': [{'action': 'check_role_permissions',
                'agent': 'AccessControlAgent',
                'data_state': None,
                'reason': "Role 'clinician' is authorized",
                'request_type': 'patient_triage',
                'result': 'granted',
                'timestamp': '2025-12-17T20:12:34.831134',
                'user_role': 'clinician'},
               {'access_granted': True,
                'action': 'record_policy_execution',
                'agent': 'AuditAgent',
                'data_state': None,
                'decryption_applied': False,
                'encryption_applied': False,
                'pii_detected_count': 0,
                'regulations': ['GDPR', 'HIPAA'],
                'request_type': 'patient_triage',
                'timestamp': '2025-12-17T20:12:34.831134',
                'user_role': 'clinician'},
               {'action': 'mask_phi',
              

In [80]:
payload = {
    "input_text": "patient  Lallon Sagar suffering from allergies,what will be right medications?",
    "request_type": "patient_triage",
    "user_role": "clinician"
}

resp = requests.post(f"{BASE_URL}/api/process", json=payload)
pretty(resp)

{'access_granted': True,
 'audit_id': 'AUDIT-20251217-201536',
 'audit_log': [{'action': 'check_role_permissions',
                'agent': 'AccessControlAgent',
                'data_state': None,
                'reason': "Role 'clinician' is authorized",
                'request_type': 'patient_triage',
                'result': 'granted',
                'timestamp': '2025-12-17T20:15:36.906649',
                'user_role': 'clinician'},
               {'access_granted': True,
                'action': 'record_policy_execution',
                'agent': 'AuditAgent',
                'data_state': None,
                'decryption_applied': False,
                'encryption_applied': False,
                'pii_detected_count': 0,
                'regulations': ['GDPR', 'HIPAA'],
                'request_type': 'patient_triage',
                'timestamp': '2025-12-17T20:15:36.906649',
                'user_role': 'clinician'},
               {'action': 'mask_phi',
              

ENABLE COMPLIANCE

In [9]:
resp = requests.post(f"{BASE_URL}/api/compliance/enable")
pretty(resp)


{'enabled': True,
 'enforced_regulations': ['GDPR', 'HIPAA', 'PCI-DSS', 'AI Act'],
 'message': 'Compliance enforcement enabled',
 'updated_at': '2025-12-17T08:54:26.153096'}


COMPLIANT MODE (FULL PIPELINE)

In [59]:
payload = {
    "input_text": "Patient John Doe, age 54, reports chest pain and shortness of breath",
    "request_type": "patient_triage",
    "user_role": "clinician"
}

resp = requests.post(f"{BASE_URL}/api/process", json=payload)
pretty(resp)


{'access_granted': True,
 'audit_id': 'AUDIT-20251217-173932',
 'audit_log': [{'action': 'check_role_permissions',
                'agent': 'AccessControlAgent',
                'data_state': None,
                'reason': "Role 'clinician' is authorized",
                'request_type': 'patient_triage',
                'result': 'granted',
                'timestamp': '2025-12-17T17:39:32.949837',
                'user_role': 'clinician'},
               {'action': 'encrypt_pii_at_rest',
                'agent': 'EncryptionAgent',
                'data_state': None,
                'encryption_applied': True,
                'pii_count': 0,
                'plaintext_preserved': False,
                'timestamp': '2025-12-17T17:39:32.949837'},
               {'access_granted': True,
                'action': 'record_policy_execution',
                'agent': 'AuditAgent',
                'data_state': None,
                'decryption_applied': False,
                'encryption_a

Cell 10 — Check Access Only

In [19]:
payload = {
    "user_role": "clinician",
    "request_type": "PatientTriage"
}

resp = requests.post(f"{BASE_URL}/api/check-access", json=payload)
pretty(resp)


{'detail': [{'input': {'request_type': 'PatientTriage',
                       'user_role': 'clinician'},
             'loc': ['body', 'user_id'],
             'msg': 'Field required',
             'type': 'missing'},
            {'input': {'request_type': 'PatientTriage',
                       'user_role': 'clinician'},
             'loc': ['body', 'patient_id'],
             'msg': 'Field required',
             'type': 'missing'}]}


Cell 11 — Mask PII

In [20]:
payload = {
    "text": "Patient John Doe lives at 221B Baker Street"
}

resp = requests.post(f"{BASE_URL}/api/mask-pii", json=payload)
pretty(resp)


{'masked_text': 'Patient [NAME] lives at 221B Baker Street',
 'message': 'Detected and masked 1 PII entities',
 'original_text': 'Patient John Doe lives at 221B Baker Street',
 'pii_count': 1,
 'pii_detected': [{'masked': '[NAME]', 'type': 'NAME', 'value': 'John Doe'}],
 'success': True}


Cell 12 — Sanitize Output

In [21]:
payload = {
    "text": "Patient John Doe diagnosis is myocardial infarction"
}

resp = requests.post(f"{BASE_URL}/api/sanitize-output", json=payload)
pretty(resp)


{'message': 'Output sanitized. Safe to store: False',
 'original_output': 'Patient John Doe diagnosis is myocardial infarction',
 'pii_detected_in_output': [{'masked': '[NAME]',
                             'type': 'NAME',
                             'value': 'John Doe'}],
 'safe_to_store': False,
 'sanitized_output': 'Patient [NAME] diagnosis is myocardial infarction',
 'success': True}


Cell 13 — Get Audit Logs

In [23]:
resp = requests.get(f"{BASE_URL}/api/audit/logs", params={"limit": 5})
pretty(resp)


{'compliance_enabled': True,
 'logs': [],
 'timestamp': '2025-12-17T09:18:24.127357',
 'total_logs': 0}


List Policies

In [24]:
resp = requests.get(f"{BASE_URL}/api/policies")
pretty(resp)


[{'enabled': True,
  'file_name': 'artificial_intelligence_act.json',
  'name': 'Artificial Intelligence Act',
  'path': 'C:\\Users\\GenAICHNSIRUSR74\\Documents\\COMPLIANCE_AI_SYSTEM\\policy_generator\\output_policies_AI\\artificial_intelligence_act.json',
  'regulation_type': 'artificial_intelligence_act'},
 {'enabled': True,
  'file_name': 'california_consumer_privacy_act_of_2018.json',
  'name': 'California Consumer Privacy Act Of 2018',
  'path': 'C:\\Users\\GenAICHNSIRUSR74\\Documents\\COMPLIANCE_AI_SYSTEM\\policy_generator\\output_policies_AI\\california_consumer_privacy_act_of_2018.json',
  'regulation_type': 'california_consumer_privacy_act_of_2018'},
 {'enabled': True,
  'file_name': 'digital_personal_data_protection_act,_2023.json',
  'name': 'Digital Personal Data Protection Act, 2023',
  'path': 'C:\\Users\\GenAICHNSIRUSR74\\Documents\\COMPLIANCE_AI_SYSTEM\\policy_generator\\output_policies_AI\\digital_personal_data_protection_act,_2023.json',
  'regulation_type': 'digital_

Cell 15 — Disable Compliance

In [ ]:
resp = requests.post(f"{BASE_URL}/api/compliance/disable")
pretty(resp)


Cell 16 — Verify Bypass Again

In [ ]:
payload = {
    "input_text": "Patient Jane Doe has fever",
    "request_type": "triage",
    "user_role": "clinician"
}

resp = requests.post(f"{BASE_URL}/api/process", json=payload)
pretty(resp)
